### State Schema With DataClasses
When we define a LangGraph StateGraph, we use a state schema.
The state schema represents the structure and types of data that our graph will use.
All nodes are expected to communicate with that schema.
LangGraph offers flexibility in how you define your state schema, accommodating various Python types and
validation approaches!


we can use the TypedDict class from python's typing module.
It allows you to specify keys and their corresponding value types.
But, note that these are type hints.
They can be used by static type checkers (like mypy) or IDEs to catch potential type-related errors before the code is run.
But they are not enforced at runtime!



In [ ]:
from typing_extensions import TypedDict
from typing import Literal

class TypedDictState(TypedDict):
    name : str
    game : Literal["cricket","badminton"]


In [ ]:
import random

def decide_play(state:TypedDictState) -> Literal["cricket","badminton"]:
    if random.random() > 0.5:
        return "cricket"
    else:
        return "badminton"

def play_game(state:TypedDictState):
    print("---- Play game node has been called ----")
    return {"name":state["name"] + " want to play "}

def cricket(state:TypedDictState):
    print(" --- cricket node has been called --- ")
    return {"name":state["name"] + "cricket","game":"cricket"}

def badminton(state:TypedDictState):
    print(" --- badminton node has been called --- ")
    return {"name":state["name"] + "badminton","game":"badminton"}

In [ ]:
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END

builder = StateGraph(TypedDictState)
builder.add_node("play_game",play_game)
builder.add_node("cricket",cricket)
builder.add_node("badminton",badminton)

builder.add_edge(START,"play_game")
builder.add_conditional_edges("play_game",decide_play)
builder.add_edge("cricket",END)
builder.add_edge("badminton",END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"name":"ishank"})

In [ ]:
graph.invoke({"name":123456})    # this not give error although it is not string

### Dataclasses
Python's dataclasses provide another way to define structured data.
Dataclasses offer a concise syntax for creating classes that are primarily used to store data.

In [ ]:
# DataClass

from dataclasses import dataclass

@dataclass
class DataClassState:
    name:str
    game:Literal["badminton","cricket"]

In [ ]:
def play_game(state:DataClassState):
    print("---- Play game node has been called ----")
    return {"name":state.name + " want to play "}

def cricket(state:DataClassState):
    print(" --- cricket node has been called --- ")
    return {"name":state.name + "cricket","game":"cricket"}

def badminton(state:DataClassState):
    print(" --- badminton node has been called --- ")
    return {"name":state.name + "badminton","game":"badminton"}

In [ ]:
import random

def decide_play(state:DataClassState) -> Literal["cricket","badminton"]:
    if random.random() > 0.5:
        return "cricket"
    else:
        return "badminton"

In [ ]:
builder = StateGraph(DataClassState)
builder.add_node("play_game",play_game)
builder.add_node("cricket",cricket)
builder.add_node("badminton",badminton)

builder.add_edge(START,"play_game")
builder.add_conditional_edges("play_game",decide_play)
builder.add_edge("cricket",END)
builder.add_edge("badminton",END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(DataClassState(name="Ishank",game="badminton"))

In [ ]:
graph.invoke(DataClassState(name=123123,game="badminton"))          # this also dont implement type safety 